# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [2]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
#пробежимся глазами по данным
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

Сразу в глаза бросается проблема с колонкой `days_employed`. 
- Во первых, дни не могут быть вещественным числом
- Во-вторых, не могут быть отрицательными. 
*Тут похоже есть проблема с выгрузкой и про это стоит создать тикет человеку, который эти данные формировал. Но, предварительно, эти данные для анализа нам не нужны.*

Имеет смысл переименовать:
- `dob_years` в `full_years`
- `children` в `children_count`, все же это не бинарный признак
- `education` в `education_level`

В столбце `education` регистр букв разный, необходимо привести к одному виду перед анализом

Доход `total_income` вряд ли интересен с точностью до копейки. Возможно, имеет смысл перевести в int

<font color='green'>Данные загружены и подробно изучены, отлично подмечены основные проблемы. Использованы подходящие методы для первичного изучения данных.</font>

## Шаг 2. Предобработка данных

### Заголовки таблицы

In [4]:
# переименуем ряд заголовков для более удобной работы с ними
df.set_axis(['children_count', 'days_employed', 'full_years', 'education_level', 'education_id', 'family_status', 'family_status_id', 'gender', 'income_type', 'debt', 'total_income', 'purpose'], axis = 'columns', inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children_count      21525 non-null int64
days_employed       19351 non-null float64
full_years          21525 non-null int64
education_level     21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Один регистр для названий категорий

In [5]:
# приведем сразу все категории к одному регистру для более удобного анализа
df['education_level'] = df['education_level'].str.lower()
df['income_type'] = df['income_type'].str.lower()
df['family_status'] = df['family_status'].str.lower()
df['purpose'] = df['purpose'].str.lower()

# вывод, чтобы проверить результат
df.head()

,children_count,days_employed,full_years,education_level,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Обработка пропусков

In [6]:
# для начала поищем пустые значения
df.isnull().sum()

children_count         0
days_employed       2174
full_years             0
education_level        0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [7]:
# посмотрим, что это за пустые значения
df[df['total_income'].isnull()].tail()

,children_count,days_employed,full_years,education_level,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21489,2,NaN,47,среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,высшее,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости
21510,2,NaN,28,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,приобретение автомобиля


In [8]:
# Заполним возможные данные total_income
# Будем использовать медиану, но не будем использовать "среднюю температуру по всей больнице"
# Чтобы данные были более точны, буду использовать уровень образования и род деятельности 

import math #нужен для проверки NaN

# функция, что будет вычислять медиану для конкретного набора по уровню образования и типа работы
# в случае, если среднее не может быть вычислено (не найдется в основных значениях группы параметров)
# берем среднее по одному - сперва по уровню образования, потом уже по типу сотрудника
def calc_median_by_education_and_income(df, education_level, income_type):
    # ориентируемся на оба параметра
    median = df[(df['education_level'] == education_level) & (df['income_type'] == income_type)]['total_income'].median()
    # но т.к. возможна уникальная ситуация, где нет совпадения для двух параметров, предусмотрим рассчет по 1му
    if math.isnan(median):
        median = df[df['education_level'] == education_level]['total_income'].median()
        if math.isnan(median):
            median = df[df['income_type'] == income_type]['total_income'].median()    
    return median

# идем по всем уровням образования и типам сотрудников, что встречаются только для пустых значений
# и проставляем только для них медиану, соответствующую их группе
for education_level in df[df['total_income'].isnull()]['education_level'].unique():
    for income_type in df[df['total_income'].isnull()]['income_type'].unique():
        median = calc_median_by_education_and_income(df, education_level, income_type)
        searchRow = (df['education_level'] == education_level) & (df['income_type'] == income_type) & (df['total_income'].isnull())
        df.loc[searchRow, 'total_income'] = df.loc[searchRow, 'total_income'].fillna(median)

# столбец с днями трудового стажа сожержит заводомо ложную информацию 
# (даже при переводе ее в дни, получается, что трудовой стаж может быть больше возраста)
# к тому же наличие отрицательных значений, с которыми тоже особо ничего не сделаешь
# Помимо этого, для наших целей данный столбец не нужен, а значит его можно удалить
del df['days_employed']

# проверяем, что все прошло хорошо и в income не осталось пустых значений
df.isnull().sum()

children_count      0
full_years          0
education_level     0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Вывод

Несуществующим значениям дохода мы выставили медиану, но не всей выборки, а только по конкретным категориям, к которым люди относились. Это позволит получить более точную информацию и снизить возможную статистическую погрешность.

Пропуски столбца `days_employed` мы не учитываем т.к. они не несут для нас значимой информации.

Если говорить о причинах возникновения пропусков, скорее всего они связаны с тем, что у людей нет подтвержденного дохода и стажа (работа в черную и тд). Однако сейчас у нас нет возможности уточнить этот момент у коллег, делающих выгрузку.




### Замена типа данных

In [9]:
# Заменим тип total_income на int, т.к. точность до копеек нам сейчас также не интересна
# поскольку нам нужно конкретный целочисленный тип, а не вещественный, используем именно astype
df['total_income'] = df['total_income'].astype('int')

# проверим, что все сделали верно
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 11 columns):
children_count      21525 non-null int64
full_years          21525 non-null int64
education_level     21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(6), object(5)
memory usage: 1.8+ MB


### Вывод

Поскольку для наших целей точность до копейки не важна, мы в приводим сумму к целому. 
К тому же, в данном столбце речь шла даже не о копейках, т.к. были заполнены больше двух знаков после запятой.

Приведение к целому числу также позволит нам выделять одних и тех же людей, но с разными целями по кредитам.

### Обработка дубликатов

In [10]:
# проверим, есть ли дубликаты в таблице с помощью метода duplicated
# используем именно его, т.к. чтобы проверить через value_counts() нужно будет создавать отдельную Series, 
# которая в дальнейшем не пригодится
df.duplicated().sum()

71

In [11]:
# удалим дубликаты
df.drop_duplicates(subset=None, keep="first", inplace=True)
# и проверим, что все хорошо
df.duplicated().sum()

0

### Вывод

Дубликатов оказалось не так много и их удалось быстро удалить.
Каких-то злостных нарушителей, т.е. повторения одного и того же не было. Возможно, было две-три одобренных заявки от одного человека либо просто человеческий фактор.

### Обработка других ошибок в данных

Дополнительно поизучав данные на уровне подсчета уникальных значений (`value_counts()`) для каждого столбца, был выявлен еще ряд проблем:
- `children_count` содержит значение 20 и -1, хотя отрицательного числа быть не может и 20 сильно выделяется на фоне остальной выборки
- `full_years` содержит нулевой возраст
- `gender` содержит пол XNA
- `purpose` содержит опечатку - ремонт жильЮ

Про возможные причины и действия:
- значение 20 в количестве детей скорее всего опечатка, когда набирая 2, случайно задели ноль
- значение -1 могло означать раннее отсуствие детей либо их наличие, но почеуму-то выгруженное, как отрицательное число. Т.к. по `df[df['children_count'] == -1]['family_status'].value_counts()` можно убедиться, что большая часть этих людей находится или находилась в браке, остановимся на втором варианте.
- нулевой возраст быть не может, скорее всего он нам просто неизвестен. Т.к. в нашем случае вопросов касательно возраста нет, ничего не делаем.
- поскольку пол XNA только один, а в наших данных большинство женщины - заменим значением по умолчанию - F
- учитывая количество строк, ошибка в самой категории на сайте или системе. У себя исправим, ответственных людей предупредим.

Исправим эти моменты.

In [12]:
# исправим опечатку в purpose
df.loc[df['purpose'] == 'ремонт жилью', 'purpose'] = 'ремонт жилья'
# проверим, что все хорошо
df.loc[df['purpose'] == 'ремонт жилью']['purpose'].count()

0

In [13]:
# исправим значение пола
df.loc[df['gender'] == 'XNA', 'gender'] = 'F'
# проверим, что все хорошо
df.loc[df['gender'] == 'XNA']['gender'].count()

0

In [14]:
# исправим значения 20 для возраста согласно нашей гипотезе
df.loc[df['children_count'] == 20, 'children_count'] = 2
# проверим, что все хорошо
df.loc[df['children_count'] == 20]['children_count'].count()

0

In [15]:
# исправим значение -1 для количества детей согласно нашей гипотезы (т.е. заменим на 1)
df.loc[df['children_count'] == -1, 'children_count'] = 1
# проверим, что все хорошо
df.loc[df['children_count'] == -1]['children_count'].count()

0

### Вывод

В данных встречаются артефакты и нам стоит об этом предупредить тех, кто делал выгрузку.
Однако их количество вряд ли отразится на статических результатах исследования. Тем не менее, для большей точности, ключевые моменты мы исправили.

### Лемматизация

In [16]:
# выделим леммы в значениях столбца с целями получения кредита

#подключим PyMystem
from pymystem3 import Mystem
m = Mystem()

# получим леммы для каждой причины, чтобы дальше проанализировать
# используем словарь, а не список, чтобы дальше это использовать для категоризации
lemmas_dict = dict()
for purpose in df['purpose'].unique():
    lemmas_dict[purpose] = m.lemmatize(purpose)
    
# проанализируем леммы - удалим пробелы, выделим самые часто встречаемые слова
# отсортируем по частоте встречания слов, чтобы получить словарь основных причин
# те слова, что встречаются только один раз, удалим

#для начала проанализируем частоту встречания слов по всем причинам кредита
lemmas_count = dict()
for key, lemmas in lemmas_dict.items():
    for lemma in lemmas:
        if lemma != ' ' and lemma != '\n':
            if not lemma in lemmas_count:
                lemmas_count[lemma] = 1
            else:
                lemmas_count[lemma] += 1
                
#выведем топ встречаемых слов                
for k in sorted(lemmas_count, key=lemmas_count.get, reverse=True):
    if lemmas_count[k] > 2:
        print(k, lemmas_count[k])

покупка 10
недвижимость 10
автомобиль 9
образование 9
жилье 7
с 5
операция 4
на 4
свой 4
свадьба 3
строительство 3
получение 3
высокий 3


### Вывод

По данным видно, что у нас присутствуют в основном 4 типа целей:
- Недвижимость (при этом недвижимость может быть представлена как жилье)
- Автомобиль
- Образование
- Свадьба

А значит леммы позволят нам поделить все цели кредитов по соответствующим группам.

In [17]:
# Подсчет через коллекции
from collections import Counter 
lemmas_list = list()
for purpose in df['purpose'].unique():
    for lemma in m.lemmatize(purpose):
        lemmas_list.append(lemma)
        
cnt = Counter(lemmas_list)
cnt = sorted(cnt, key=cnt.get, reverse=True) 

cnt

[' ',
 '\n',
 'покупка',
 'недвижимость',
 'автомобиль',
 'образование',
 'жилье',
 'с',
 'операция',
 'на',
 'свой',
 'свадьба',
 'строительство',
 'получение',
 'высокий',
 'дополнительный',
 'для',
 'коммерческий',
 'жилой',
 'заниматься',
 'сделка',
 'приобретение',
 'сыграть',
 'проведение',
 'семья',
 'собственный',
 'подержать',
 'со',
 'подержанный',
 'профильный',
 'сдача',
 'ремонт']

In [43]:
#код ревьюера
Counter(m.lemmatize(' '.join(df['purpose']))).most_common(15)

[(' ', 55023),
 ('недвижимость', 6351),
 ('покупка', 5897),
 ('жилье', 4460),
 ('автомобиль', 4306),
 ('образование', 4013),
 ('с', 2918),
 ('операция', 2604),
 ('свадьба', 2324),
 ('свой', 2230),
 ('на', 2222),
 ('строительство', 1878),
 ('высокий', 1374),
 ('получение', 1314),
 ('коммерческий', 1311)]

### Категоризация данных

In [18]:
# сделаем столбец с причинами для взятия кредита по категориям, что мы выделили на этапе лемматизации

# напишем функцию, по которой заполним столбец
def make_true_purpose(table_purpose):
    #lemmas_dict мы сохраняли ранее
    purpose_lemmas = lemmas_dict[table_purpose]
    if 'недвижимость' in purpose_lemmas or 'жилье' in purpose_lemmas:
        return 'недвижимость'
    
    if 'автомобиль' in purpose_lemmas:
        return 'автомобиль'
    
    if 'образование' in purpose_lemmas:
        return 'образование'
    
    if 'свадьба' in purpose_lemmas:
        return 'свадьба'
    
    return ''

df['true_purpose'] = df['purpose'].apply(make_true_purpose)
    
#проверим, что никто не остался без категории
df[df['true_purpose'] == '']

,children_count,full_years,education_level,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,true_purpose


In [19]:
# посмотрим как распределились цели кредитов
df['true_purpose'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: true_purpose, dtype: int64

In [20]:
# Поскольку больше двух детей данных в принципе не очень много, имеет смысл объеденить их в одну категорию
def make_children_category(children_count):
    if children_count == 1:
        return '1 ребенок'
    if children_count == 2:
        return '2 ребенка'
    if children_count >= 3:
        return 'Много детей'
    return 'Нет детей'

df['children_category'] = df['children_count'].apply(make_children_category)

# проверим, что все ок
df['children_category'].value_counts()

Нет детей      14091
1 ребенок       4855
2 ребенка       2128
Много детей      380
Name: children_category, dtype: int64

In [21]:
# уровень дохода также нужно определять конкретными категориями, иначе никакого анализа не получится
# к тому же в нашем случае разброс от 20 тысяч до 2+ миллионов
# будем использовать следующие: до 50, от 50 до 100, от 100 до 150, от 150 до 200, от 200 до 500, от 500 и выше

def make_income_category(total_income):
    if total_income < 50000:
        return 'До 50000'
    if 50000 <= total_income < 100000:
        return 'От 50 до 100'
    if 100000 <= total_income < 150000:
        return 'От 100 до 150'
    if 150000 <= total_income < 200000:
        return 'От 150 до 200'
    if 200000 <= total_income < 500000:
        return 'От 200 до 500'
    return 'Больше 500'


df['income_category'] = df['total_income'].apply(make_income_category)

# проверим, что все ок
df['income_category'].value_counts()

От 100 до 150    6971
От 200 до 500    5031
От 150 до 200    4767
От 50 до 100     4091
До 50000          372
Больше 500        222
Name: income_category, dtype: int64

### Вывод

Из любопытного:
- Судя по уровню дохода, кредит не особо интересен тем, кто зарабатывает очень много, как и тем, кто зарабатывает мало. А вот остальным (в диапазоне от 50 до 500) он интересен. При чем ближе к середине этих групп наиболее интересен.
- Кредит не бояться брать те, у кого нет детей, а чем детей больше - тем меньше желающих
- Больше всего интересуют именно кредиты, связанные с недвижимостью

### Категоризация данных с помощью квантилей (добавлено после ревью)

In [22]:
# посмотрим отсечки, что предложит нам describe
df['total_income'].describe()

count    2.145400e+04
mean     1.654530e+05
std      9.828866e+04
min      2.066700e+04
25%      1.075155e+05
50%      1.437075e+05
75%      1.983070e+05
max      2.265604e+06
Name: total_income, dtype: float64

In [23]:
# в более удобном виде через quantile
df['total_income'].quantile([.25, .5, .75])

0.25    107515.5
0.50    143707.5
0.75    198307.0
Name: total_income, dtype: float64

In [24]:
def make_income_category_by_quantile(total_income):
    if total_income < 107515.5:
        return 'До 107515'
    if 107515.5 <= total_income < 143707.5:
        return 'От 107515 до 143707'
    if 143707.5 <= total_income < 198307:
        return 'От 143707 до 198307'
    return 'Больше 198307'

df['income_quantile_category'] = df['total_income'].apply(make_income_category_by_quantile)

# проверим, что все ок
df['income_quantile_category'].value_counts()

До 107515              5364
Больше 198307          5364
От 107515 до 143707    5363
От 143707 до 198307    5363
Name: income_quantile_category, dtype: int64

In [56]:
# построим сводную таблицу для ответа на вопрос
data_pivot = df.pivot_table(index=['income_quantile_category'], values=["debt"], aggfunc=['sum', 'count', make_proportion])
# сортируем, чтобы сразу видеть у кого ситуация лучше
data_pivot = data_pivot.sort_values(by=('make_proportion', 'debt'))
data_pivot

,sum,count,make_proportion
,debt,debt,debt
income_quantile_category,,,
Больше 198307,373,5364,6.95%
До 107515,427,5364,7.96%
От 143707 до 198307,456,5363,8.5%
От 107515 до 143707,485,5363,9.04%


### Вывод

На самом деле, распределение по количеству измерений, оказалось
- Более репрезентативным (т.к. доверия этим данным больше за счет количества измерений)
- Но, возможно, менее показательным

Здесь также видно, что чем больше денег и чем меньше, тем меньше возникает просрочек. Однако тот факт, что до 50к просрочек еще меньше и что количество заявок, как при низком доходе, так и при высоком, маленькое - по этим данным не видно.

Также разница между конкретно этими группами оказалась не настолько очевидной и не позволяет сделать вывод о силе тенденции.
Однако, инструмент очень удобен для старта и общего анализа данных.

Эти данные мы используем в итоговом выводе

### Выделение словарей

По данным у нас напрашиваются два словаря. Более того, у нас уже предусмотрены соответствующие id
- `education_level`
- `family_status`

Помимо этого, можно вынести отдельно `gender` и `purpose`. Но `gender` не занимает много места и не сильно нужен нам для анализа, по сути столбец вовсе можно удалить. А `purpose` не отражает реальных категорий, поэтому мы выделяли типы отдельно, поэтому словарь для него выйдет избыточным.

Поэтому остановимся на словарях для `education_level` и `family_status`

In [47]:
# выделим словарь для education
education_dict = df[['education_id', 'education_level']]
# удалим дубликаты из словаря
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
# проверим содержимое
education_dict.head(10)

,education_id,education_level
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [48]:
# аналогичным образом поступим для family_status

# выделим словарь для family_status
family_status_dict = df[['family_status_id', 'family_status']]
# удалим дубликаты из словаря
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
# проверим содержимое
family_status_dict.head(10)

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


In [49]:
# т.к. словари мы сохранили, а id проставлены, удалим столбцы education_level и family_status из основной таблицы
del df['education_level']
del df['family_status']

#посмотрим, остались ли столбцы
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21454 entries, 0 to 21524
Data columns (total 13 columns):
children_count              21454 non-null int64
full_years                  21454 non-null int64
education_id                21454 non-null int64
family_status_id            21454 non-null int64
gender                      21454 non-null object
income_type                 21454 non-null object
debt                        21454 non-null int64
total_income                21454 non-null int64
purpose                     21454 non-null object
true_purpose                21454 non-null object
children_category           21454 non-null object
income_category             21454 non-null object
income_quantile_category    21454 non-null object
dtypes: int64(6), object(7)
memory usage: 2.3+ MB


### Шаг 3. Ответьте на вопросы

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [50]:
# функция, что посчитает нам отношение в процентах, будем использовать для ответов на все вопросы
def make_proportion(pdSerises):
    return str(round((pdSerises.sum() / pdSerises.count()) * 100, 2)) + '%'

# построим сводную таблицу для ответа на вопрос
data_pivot = df.pivot_table(index=['children_category'], values=["debt"], aggfunc=['sum', 'count', make_proportion])
# сортируем, чтобы сразу видеть у кого ситуация лучше
data_pivot = data_pivot.sort_values(by=('make_proportion', 'debt'))
data_pivot

,sum,count,make_proportion
,debt,debt,debt
children_category,,,
Нет детей,1063,14091,7.54%
Много детей,31,380,8.16%
1 ребенок,445,4855,9.17%
2 ребенка,202,2128,9.49%


### Вывод

Когда детей нет, вероятность возникновения задолженности чуть ниже, чем если дети есть. Что любопытно, когда детей много, задолженность возникает примерно на том же уровне. В остальных случаях вероятность просрочки выше.

Но на самом деле разница не такая большая.
Детальные выводы помогла бы сделать медиана по сумме запрошенных кредитов, но этих данных пока нет.

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [51]:
# поскольку мы до этого удаляли из основных данных текстовые значения family_status, тут необходимо их вернуть
df_with_family_status = df.merge(family_status_dict, on='family_status_id', how='left')

# построим сводную таблицу для ответа на вопрос (по фрейму со значениями из словаря)
data_pivot = df_with_family_status.pivot_table(index=['family_status'], values=["debt"], aggfunc=['sum', 'count', make_proportion])
# сортируем, чтобы сразу видеть у кого ситуация лучше
data_pivot = data_pivot.sort_values(by=('make_proportion', 'debt'))
data_pivot

,sum,count,make_proportion
,debt,debt,debt
family_status,,,
вдовец / вдова,63,959,6.57%
в разводе,85,1195,7.11%
женат / замужем,931,12339,7.55%
гражданский брак,388,4151,9.35%
не женат / не замужем,274,2810,9.75%


### Вывод

Интересно, что вдовец/вдова и люди в разводе имеют наименьший риск просрочки. Для меня это даже удивительно.
Возможно, это связно с тем, что кредит берется более осознанно.

А вот самая "опасная" категория - неженатые и состоящие в гражданском браке.
Чем серьезней отношения - тем меньше шанс просрочки?

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [52]:
# построим сводную таблицу для ответа на вопрос
data_pivot = df.pivot_table(index=['income_category'], values=["debt"], aggfunc=['sum', 'count', make_proportion])
# сортируем, чтобы сразу видеть у кого ситуация лучше
data_pivot = data_pivot.sort_values(by=('make_proportion', 'debt'))
data_pivot

,sum,count,make_proportion
,debt,debt,debt
income_category,,,
До 50000,23,372,6.18%
Больше 500,14,222,6.31%
От 200 до 500,352,5031,7.0%
От 50 до 100,331,4091,8.09%
От 150 до 200,405,4767,8.5%
От 100 до 150,616,6971,8.84%


### Вывод

Тут мы видим, что меньше всего просрочка у самых обеспеченных людей и у тех, у кого денег особо и нет. При чем ни первые, ни вторые не особо любят оформлять заявки на кредит.

А вот самыя "опасная" группа доходов - у условного среднего класса.

### Как разные цели кредита влияют на его возврат в срок?

In [53]:
# построим сводную таблицу для ответа на вопрос
data_pivot = df.pivot_table(index=['true_purpose'], values=["debt"], aggfunc=['sum', 'count', make_proportion])
# сортируем, чтобы сразу видеть у кого ситуация лучше
data_pivot = data_pivot.sort_values(by=('make_proportion', 'debt'))
data_pivot

,sum,count,make_proportion
,debt,debt,debt
true_purpose,,,
недвижимость,782,10811,7.23%
свадьба,186,2324,8.0%
образование,370,4013,9.22%
автомобиль,403,4306,9.36%


### Вывод

Выгоднее всего давать кредиты на недвижимость, а автомобиль и образование - в зоне риска.
Хоть абсолютная разница не такая большая, относительно друг друга она составляет порядка 22%, что уже не мало

### Шаг 4. Общий вывод

Самый идеальный заемщик - это человек без детей, состоящий или состоявший в официальных отношениях, имеющий уровень дохода выше среднего и берущий деньги на недвижимость.

"Разброс" просроченных выплат по каждой из категорий примерно одинаковый. В среднем, это диапазоны от 7 до 9 процентов. 
В остальном разница в абсолютных цифрах не такая большая. Однако в относительных разница составляет в среднем 2 процента, что означает разницу в вероятности просрочки платежа на 20% между более 

Детально по категориям:
- Влияние наличия детей на вероятность просрочки платежа
 - По наличию детей
   1. Нет детей - 7.54%
   2. Много детей - 8.16%
   3. 1 ребенок - 9.17%
   4. 2 ребенка - 9.49%
 - Абсолютная разница между минимальным и максимальным значением - 1.95%
 - Относительная разница между минимальным и максимальным значением - 20.54%
- Влияние семейного положения на вероятность просрочки платежа
 - По семейному положению
   1. вдовец / вдова - 6.57%
   2. в разводе - 7.11%
   3. женат / замужем - 7.55%
   4. гражданский брак - 9.35%
   5. не женат / не замужем - 9.75%
 - Абсолютная разница между минимальным и максимальным значением - 3.18%
 - Относительная разница между минимальным и максимальным значением - 32,6%
- Влияние уровня дохода на просрочку платежа
 - По уровню дохода
   1. До 50000 - 6.18%
   2. Больше 500 - 6.31%
   3. От 200 до 500	- 7.0%
   4. От 50 до 100 - 8.09%
   5. От 150 до 200 - 8.5%
   6. От 100 до 150	- 8.84%
 - Абсолютная разница между минимальным и максимальным значением - 2.66%
 - Относительная разница между минимальным и максимальным значением - 30.09%
- Влияние цели кредита на вероятность возврата кредита в срок
 - По цели кредита
   1. недвижимость - 7.23%
   2. свадьба - 8.0%
   3. образование - 9.22%
   4. автомобиль - 9.36%
 - Абсолютная разница между минимальным и максимальным значением - 2.13%
 - Относительная разница между минимальным и максимальным значением - 22.76%

При этом, нужно отметить, что размер выборки уровня дохода для значений "до 50000" и "больше 500000" мал. Из этого можно сделать вывод, что люди со сверхдоходами не так часто нуждаются в кредитах, а также и то, что люди с малым количеством дохода также опасаются их брать. Однако же, это означает, что итоговые данные по этим выборкам могут быть статистически неточными.

Поэтому, расширим наш вывод. Используя квантили для распределения по группам уровня дохода мы получаем следующие данные:
- Влияние уровня дохода по равным квантилям
 - По уровню дохода
   1. Больше 198307 - 6.95%
   2. До 107515 - 7.96%
   3. От 143707 до 198307 - 8.5%
   4. От 107515 до 143707 - 9.04%
 - Абсолютная разница между минимальным и максимальным значением - 2.09%
 - Относительная разница между минимальным и максимальным значением - 23.12%
 
Таким образом, несмотря на схожий в процентном отношении разброс значений, наибольшее влияние на вероятность просрочки показывает именно семейное положение. Во вторую очередь, следует обращать внимание на уровень дохода. Наименьшее влияние оказывают цели кредита и наличие детей.